In [1]:
!pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 KB ? eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 21.2 MB/s eta 0:00:00
     ------------------------------------- 439.2/439.2 KB 26.8 MB/s eta 0:00:00
     --------------------------------------- 23.2/23.2 MB 17.7 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 20.0 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 7.3 MB/s eta 0:00:00
     ------------------------------------- 895.9/895.9 KB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 14.7 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 15.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 13.4 MB/s eta 0:00:00
     ---------------------------------------- 64.5/6

You should consider upgrading via the 'C:\Users\a.saratovtsev\hse\venv\Scripts\python.exe -m pip install --upgrade pip' command.


1\. Describe in writing why cross entropy performs better as a loss function then mean squared error.

2\. Describe in writing why dropout allows to reduce overfitting.

3\. Using only dense layers and dropouts try to improve performance of the network predicting handwritten digits. 

In [9]:
import tensorflow as tf

# This initialization code is required due to an error 
# "NotFoundError: No algorithm worked"
# when using Conv2D
# Probabliy due to problems with cuda 11.
# Remove this when fixed
# https://github.com/tensorflow/tensorflow/issues/43174
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.utils import to_categorical as to_categ

In [10]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\a.saratovtsev\hse\venv\lib\site-packages\tensorflow\python\client\session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [11]:
# Returns tuple of numpy arrays: (x_train, y_train), (x_test, y_test)
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print("Image labels example: ", y_train[:20])

Image labels example:  [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


In [12]:
import numpy as np

def extract01(X_data, y_data):
    """Extract images with 0 or 1 only"""
    X_data01 = []
    y_data01 = []
    for X, y in zip(X_data, y_data):
        if y <= 1:
            X_data01.append(X)
            y_data01.append(y)
    return np.array(X_data01), np.array(y_data01)

X_train01, y_train01 = extract01(X_train, y_train)
X_test01, y_test01 = extract01(X_test, y_test)

print("train ", X_train01.shape)
print("test ", X_test01.shape)

train  (12665, 28, 28)
test  (2115, 28, 28)


In [13]:
input_shape = X_train.shape[1:]

dropout_rate = 0.02

model_mnist_C = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda data: data / 256.0, input_shape=input_shape),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(input_shape[0]*3, activation='relu'), tf.keras.layers.Dropout(rate=dropout_rate),
    tf.keras.layers.Dense(input_shape[0]*2, activation='relu'), tf.keras.layers.Dropout(rate=dropout_rate),
    tf.keras.layers.Dense(input_shape[0], activation='relu'), tf.keras.layers.Dropout(rate=dropout_rate),
    tf.keras.layers.Dense(input_shape[0], activation='relu'), tf.keras.layers.Dropout(rate=dropout_rate),
    tf.keras.layers.Dense(10) 
])

model_mnist_C.compile(optimizer=tf.keras.optimizers.Adam(),  
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

model_mnist_C.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 28, 28)            0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 84)                65940     
                                                                 
 dropout_4 (Dropout)         (None, 84)                0         
                                                                 
 dense_6 (Dense)             (None, 56)                4760      
                                                                 
 dropout_5 (Dropout)         (None, 56)                0         
                                                                 
 dense_7 (Dense)             (None, 28)               

In [14]:
hist_mnist_C = model_mnist_C.fit(X_train, to_categ(y_train), epochs=20, validation_split=0.2, verbose=2)

Epoch 1/20
1500/1500 - 5s - loss: 0.3869 - accuracy: 0.8814 - val_loss: 0.1657 - val_accuracy: 0.9529 - 5s/epoch - 3ms/step
Epoch 2/20
1500/1500 - 3s - loss: 0.1562 - accuracy: 0.9537 - val_loss: 0.1234 - val_accuracy: 0.9634 - 3s/epoch - 2ms/step
Epoch 3/20
1500/1500 - 3s - loss: 0.1169 - accuracy: 0.9651 - val_loss: 0.1090 - val_accuracy: 0.9687 - 3s/epoch - 2ms/step
Epoch 4/20
1500/1500 - 3s - loss: 0.0931 - accuracy: 0.9711 - val_loss: 0.1122 - val_accuracy: 0.9705 - 3s/epoch - 2ms/step
Epoch 5/20
1500/1500 - 3s - loss: 0.0800 - accuracy: 0.9750 - val_loss: 0.1093 - val_accuracy: 0.9695 - 3s/epoch - 2ms/step
Epoch 6/20
1500/1500 - 3s - loss: 0.0681 - accuracy: 0.9798 - val_loss: 0.1211 - val_accuracy: 0.9669 - 3s/epoch - 2ms/step
Epoch 7/20
1500/1500 - 3s - loss: 0.0614 - accuracy: 0.9816 - val_loss: 0.1206 - val_accuracy: 0.9677 - 3s/epoch - 2ms/step
Epoch 8/20
1500/1500 - 3s - loss: 0.0570 - accuracy: 0.9818 - val_loss: 0.1133 - val_accuracy: 0.9709 - 3s/epoch - 2ms/step
Epoch 9/

In [15]:
loss, acc = model_mnist_C.evaluate(X_test, to_categ(y_test))
print(f"acc={acc}, loss={loss}")

313/313 [==============================] - 1s 2ms/step - loss: 0.1085 - accuracy: 0.9742
acc=0.9742000102996826, loss=0.10846102237701416


In [16]:
input_shape = X_train.shape[1:]

dropout_rate = 0.2

model_mnist_D = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda data: data / 256.0, input_shape=input_shape),
    tf.keras.layers.Reshape((*input_shape, 1)),
    tf.keras.layers.Conv2D(4, (4, 4), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(input_shape[0]*2, activation='relu'), 
    tf.keras.layers.Dropout(rate=dropout_rate), 
    tf.keras.layers.Dense(10) 
])

model_mnist_D.compile(optimizer=tf.keras.optimizers.Adam(),  
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

model_mnist_D.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 28, 28)            0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 4)         68        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 9, 9, 4)          0         
 )                                                               
                                                                 
 flatten_2 (Flatten)         (None, 324)               0         
                                                                 
 dense_10 (Dense)            (None, 56)                18200     
                                                      

In [17]:
hist_mnist_D = model_mnist_D.fit(X_train, to_categ(y_train), epochs=15, validation_split=0.2, verbose=2)

Epoch 1/15
1500/1500 - 9s - loss: 0.4328 - accuracy: 0.8655 - val_loss: 0.1522 - val_accuracy: 0.9551 - 9s/epoch - 6ms/step
Epoch 2/15
1500/1500 - 8s - loss: 0.1748 - accuracy: 0.9468 - val_loss: 0.1080 - val_accuracy: 0.9682 - 8s/epoch - 5ms/step
Epoch 3/15
1500/1500 - 8s - loss: 0.1306 - accuracy: 0.9598 - val_loss: 0.0916 - val_accuracy: 0.9721 - 8s/epoch - 6ms/step
Epoch 4/15
1500/1500 - 8s - loss: 0.1092 - accuracy: 0.9665 - val_loss: 0.0799 - val_accuracy: 0.9756 - 8s/epoch - 5ms/step
Epoch 5/15
1500/1500 - 8s - loss: 0.0934 - accuracy: 0.9709 - val_loss: 0.0698 - val_accuracy: 0.9787 - 8s/epoch - 5ms/step
Epoch 6/15
1500/1500 - 9s - loss: 0.0846 - accuracy: 0.9738 - val_loss: 0.0704 - val_accuracy: 0.9779 - 9s/epoch - 6ms/step
Epoch 7/15
1500/1500 - 10s - loss: 0.0773 - accuracy: 0.9753 - val_loss: 0.0689 - val_accuracy: 0.9787 - 10s/epoch - 7ms/step
Epoch 8/15
1500/1500 - 10s - loss: 0.0709 - accuracy: 0.9780 - val_loss: 0.0612 - val_accuracy: 0.9817 - 10s/epoch - 7ms/step
Epoc

In [18]:
loss, acc = model_mnist_D.evaluate(X_test, to_categ(y_test))
print(f"acc={acc}, loss={loss}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.9841
acc=0.9840999841690063, loss=0.04902346059679985


In [19]:
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras import datasets, layers, models
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 113s 1us/step


In [20]:
# Converting the pixels data to float type
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
 
# Standardizing (255 is the total number of pixels an image can have)
train_images = train_images / 255
test_images = test_images / 255 

# One hot encoding the target class (labels)
num_classes = 10
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

In [21]:
model = Sequential()

model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))    # num_classes = 10

# Checking the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                      

In [22]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=64, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
782/782 [==============================] - 185s 232ms/step - loss: 1.7216 - accuracy: 0.4054 - val_loss: 1.2336 - val_accuracy: 0.5581
Epoch 2/10
782/782 [==============================] - 185s 237ms/step - loss: 1.1672 - accuracy: 0.5835 - val_loss: 1.0465 - val_accuracy: 0.6352
Epoch 3/10
782/782 [==============================] - 185s 237ms/step - loss: 1.0032 - accuracy: 0.6476 - val_loss: 1.0106 - val_accuracy: 0.6394
Epoch 4/10
782/782 [==============================] - 184s 235ms/step - loss: 0.8966 - accuracy: 0.6874 - val_loss: 0.7717 - val_accuracy: 0.7361
Epoch 5/10
782/782 [==============================] - 174s 223ms/step - loss: 0.8152 - accuracy: 0.7160 - val_loss: 0.7164 - val_accuracy: 0.7439
Epoch 6/10
782/782 [==============================] - 172s 220ms/step - loss: 0.7637 - accuracy: 0.7374 - val_loss: 0.6683 - val_accuracy: 0.7671
Epoch 7/10
782/782 [==============================] - 172s 219ms/step - loss: 0.7209 - accuracy: 0.7507 - val_loss: 0.6766 -

In [23]:
loss, acc = model.evaluate(test_images, test_labels)
print(f"acc={acc}, loss={loss}")

313/313 [==============================] - 7s 23ms/step - loss: 0.5701 - accuracy: 0.8076
acc=0.8076000213623047, loss=0.5701103806495667


Dropout regularization will ensure the following:

- The neurons can’t rely on one input because it might be dropped out at random. This reduces bias due to over-relying on one input, bias is a major cause of overfitting.

- Neurons will not learn redundant details of inputs. This ensures only important information is stored by the neurons. This enables the neural network to gain useful knowledge which it uses to make predictions.